In [1]:
from ragbooster import Generator, BingRetriever, RetrievalAugmentedModel, RAGBooster, score
from ragbooster.demo import load_wikifact_questions

questions = load_wikifact_questions('demo_data/wikifact_place_of_birth_helm.json')

validation_questions = questions[:500]
test_questions = questions[500:1000]

example_question = questions[5]
example_question

Question(text='Martin Kulha was born in', correct_answers=['Poprad'], metadata={})

In [4]:
import re 

class PlaceOfBirthGenerator(Generator):
    
    FEW_SHOT_PROMPT = "Brown was born in England\n\n"+\
        "Jerry Beck was born in New York City\n\n"+\
        "Werner Lorenz was born in Ludwigshafen\n\n"+\
        "Moritz Retzsch was born in Dresden\n\n"+\
        "Roni Rosadi was born in Bandar Lampung\n\n"    
    
    RETRIEVAL_PROMPT = "\nJerry Beck (born February 9, 1955, in New York City) is an American animation historian," +\
        " author, blogger, and video producer.Beck wrote or edited several books on classic" +\
        " American animation and classic characters.\nJerry Beck was born in New York\n\n" +\
        "Ettore Maria Fizzarotti (1916–1985) was an Italian film director and screenwriter." +\
        " Born in Naples, the son of the director Armando, he debuted as assistant director" +\
        " in the films of his father.\nEttore Maria Fizzarotti was born in Naples\n"   
    
    def __init__(self, llm):
        super().__init__(llm=llm, max_tokens=10)    
    
    def _create_prompt(self, question):        
        return f"{self.FEW_SHOT_PROMPT}\n\n{question.text}"            
    
    def _create_retrieval_prompt(self, question, retrieved_text):        
        return f"{self.RETRIEVAL_PROMPT}\n\n{retrieved_text}\n\n{question.text}"          

    def _extract_answer(self, response):
        answer = response.get_response()          

        answer = re.sub(r'[0-9]+', '', answer)
        answer = answer.strip()   

        for sep in ['\n', ',', '.']:
            if sep in answer:
                answer = answer.split(sep)[0]

        return answer.strip()  


In [20]:
from manifest import Manifest 

gpt35_client = Manifest(client_name="openai", engine="text-davinci-003",
                        cache_name="sqlite", cache_connection="demo_data/gpt35-cache.sqlite")

gpt35 = PlaceOfBirthGenerator(llm=gpt35_client)

# https://huggingface.co/togethercomputer/RedPajama-INCITE-Instruct-3B-v1
redpajama_client = Manifest(client_name = "huggingface", client_connection = "http://127.0.0.1:5689",
                            cache_name='sqlite', cache_connection="remote-huggingface-cache.sqlite")

redpajama = PlaceOfBirthGenerator(llm=redpajama_client)

In [21]:
print(f'GPT3.5 answers "{example_question.text}" with "{gpt35.generate(example_question)}"')
print(f'RedPajama3B answers "{example_question.text}" with "{redpajama.generate(example_question)}"')
print(f'Correct answers are: {example_question.correct_answers}')

GPT3.5 answers "Martin Kulha was born in" with "Prague"
RedPajama3B answers "Martin Kulha was born in" with "Prague"
Correct answers are: ['Poprad']


In [22]:
accuracy = score(test_questions, gpt35)

f'The accuracy of GPT3.5 on the test set is {accuracy}'

  0%|          | 0/500 [00:00<?, ?it/s]

'The accuracy of GPT3.5 on the test set is 0.14'

In [23]:
accuracy = score(test_questions, redpajama)

f'The accuracy of RedPajama3B on the test set is {accuracy}'

  0%|          | 0/500 [00:00<?, ?it/s]

'The accuracy of RedPajama3B on the test set is 0.1'

In [24]:
class MyBingWebsearch(BingRetriever):
    
    def __init__(self, cache_path):
        super().__init__(cache_path)
    
    def create_query(self, question):
        return question.text
    
bing_websearch = MyBingWebsearch('demo_data/bing-cache.pkl')    

In [25]:
retrieved = bing_websearch.retrieve(example_question)
for snippet, url in retrieved[:3]:
    print(url, '-', snippet, '\n')


https://www.wikilogy.com/biography/martin-kulha/ - Martin Kulha is an Ice Hockey Player. He was born in Poprad on August 07, 1976. Want to more about Him? In this article, we covered Martin Kulha's net worth, wiki, bio, career, height, weight, pics, family, affairs, car, salary, age, facts, and other details in 2023. Continue reading to discover who is Martin Kulha. 

https://www.celebsagewiki.com/martin-kulha - Martin Kulha was born on 7 August, 1976 in Poprad, Slovakia. Discover Martin Kulha's Biography, Age, Height, Physical Stats, Dating/Affairs, Family and career updates. Learn How rich is She in this year and how She spends money? Also learn how She earned most of networth at the age of 44 years old? 

https://icehockey.fandom.com/wiki/Martin_Kulha - Martin Kulha (born August 7, 1976) is a Slovak professional ice hockey player who formerly played with Sangliers Arvernes de Clermont in the FFHG Division 1. He is now a member of the Lyon Club in the French Division 3. Kulha had pre

In [26]:
for snippet, url in retrieved[:10]:
    answer = gpt35.generate_with_retrieved_context(example_question, snippet)
    print(f'GPT3.5 gives the answer "{answer}" based on {url}')    

GPT3.5 gives the answer "Poprad on August" based on https://www.wikilogy.com/biography/martin-kulha/
GPT3.5 gives the answer "Poprad" based on https://www.celebsagewiki.com/martin-kulha
GPT3.5 gives the answer "Poprad" based on https://icehockey.fandom.com/wiki/Martin_Kulha
GPT3.5 gives the answer "Poprad" based on https://biogossipy.com/martin-kulha/
GPT3.5 gives the answer "Poprad" based on https://popularbio.com/martin-kulha/
GPT3.5 gives the answer "Poprad" based on https://www.hockeydb.com/ihdb/stats/pdisplay.php?pid=57405
GPT3.5 gives the answer "Pohoří" based on https://www.myheritage.com/names/martin_kulha
GPT3.5 gives the answer "Slovakia" based on http://www.vipfaq.com/Martin%20Kulha.html
GPT3.5 gives the answer "Poprad" based on https://networthmask.com/martin-kulha/
GPT3.5 gives the answer "August th" based on https://en.wikipedia.org/wiki/Martin_Kulha


In [27]:
rag = RetrievalAugmentedModel(bing_websearch, gpt35, k=10)

print(f'GPT3.5 with retrieval augmentation gives the correct answer "{rag.generate(example_question)}"')   

GPT3.5 with retrieval augmentation gives the correct answer "Poprad"


In [28]:
gpt35_rag1 = RetrievalAugmentedModel(bing_websearch, gpt35, k=1)

accuracy = score(test_questions, gpt35_rag1)

f'The accuracy of GPT3.5 with retrieval augmentation and k=1 on the test set is {accuracy}'

  0%|          | 0/500 [00:00<?, ?it/s]

'The accuracy of GPT3.5 with retrieval augmentation and k=1 on the test set is 0.336'

In [30]:
redpajama_rag1 = RetrievalAugmentedModel(bing_websearch, redpajama, k=1)

accuracy = score(test_questions, redpajama_rag1)

f'The accuracy of RedPajama3B with retrieval augmentation and k=1 on the test set is {accuracy}'

  0%|          | 0/500 [00:00<?, ?it/s]

'The accuracy of RedPajama3B with retrieval augmentation and k=1 on the test set is 0.41'

In [37]:
gpt35_rag10 = RetrievalAugmentedModel(bing_websearch, gpt35, k=10)

accuracy_gpt35_rag10 = score(test_questions, gpt35_rag10)

f'The accuracy of GPT3.5 with retrieval augmentation and k=10 on the test set is {accuracy_gpt35_rag10}'

  0%|          | 0/500 [00:00<?, ?it/s]

'The accuracy of GPT3.5 with retrieval augmentation and k=10 on the test set is 0.498'

In [42]:
redpajama_rag10 = RetrievalAugmentedModel(bing_websearch, redpajama, k=10)

accuracy_redpajama_rag10 = score(test_questions, redpajama_rag10)

f'The accuracy of RedPajama3B with retrieval augmentation and k=10 on the test set is {accuracy_redpajama_rag10}'

  0%|          | 0/500 [00:00<?, ?it/s]

'The accuracy of RedPajama3B with retrieval augmentation and k=10 on the test set is 0.496'

In [34]:
gpt35_rag_boosted = RAGBooster(gpt35_rag10, validation_questions)

Computing validation corpus...


  0%|          | 0/500 [00:00<?, ?it/s]

Learning importance weights for data sources...
Tuning threshold for corpus pruning...
Achieved accuracy of 0.586 with a pruning threshold of 0.54092 on the validation set.


In [39]:
accuracy = score(test_questions, gpt35_rag_boosted)

f'RAGBooster boosted the accuracy of GPT3.5 with retrieval augmentation'+\
f' from {accuracy_gpt35_rag10} to {accuracy}!'

  0%|          | 0/500 [00:00<?, ?it/s]

'RAGBooster boosted the accuracy of GPT3.5 with retrieval augmentation from 0.498 to 0.532!'

In [40]:
redpajama_rag_boosted = RAGBooster(redpajama_rag10, validation_questions)

Computing validation corpus...


  0%|          | 0/500 [00:00<?, ?it/s]

Learning importance weights for data sources...
Tuning threshold for corpus pruning...
Achieved accuracy of 0.596 with a pruning threshold of 0.54764 on the validation set.


In [44]:
accuracy = score(test_questions, redpajama_rag_boosted)

f'RAGBooster boosted the accuracy of RedPajama3B with retrieval augmentation'+\
f' from {accuracy_redpajama_rag10} to {accuracy}!'

  0%|          | 0/500 [00:00<?, ?it/s]

'RAGBooster boosted the accuracy of RedPajama3B with retrieval augmentation from 0.496 to 0.528!'